# OpenAI Function Calling

The newer OpenAI Function Calling Functionality allows to to define functions which will be passed to the LLM. The LLM will identify the correct function for the request and provide parameters for the function call.



In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import openai

def chat(query):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
    )
    message = response.choices[0].message.content
    return message

In [3]:
query = "How much does pizza salami cost?"
message = chat(query)
message

'The cost of pizza salami can vary depending on the size of the pizza, the restaurant or store it is purchased from, and the region. On average, a medium-sized pizza with salami toppings can cost around $10 to $15.'

To make use of Function calling you need:

1. A function
2. A dictionary which describes the function

In [4]:
import json

def get_pizza_info(pizza_name: str):
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

In [5]:
functions = [
    {
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Salami",
                },
            },
            "required": ["pizza_name"],
        },
    }
]

In [6]:
def chat(query):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
        functions=functions, # this is new
    )
    message = response.choices[0].message
    return message

In [7]:
chat("What is the capital of france?")


ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None)

In [8]:
query = "How much does pizza salami cost?"
message = chat(query)
print(message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"pizza_name":"Salami"}', name='get_pizza_info'), tool_calls=None)


In [9]:
if message.function_call:
    pizza_name = json.loads(message.function_call.arguments).get("pizza_name")
    print(pizza_name)
    function_response = get_pizza_info(
        pizza_name=pizza_name
    )
    print(function_response)

Salami
{"name": "Salami", "price": "10.99"}


In [10]:
second_response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": query},
        message,
        {
            "role": "function",
            "name": "get_pizza_info",
            "content": function_response,
        },
    ],
)
second_response

ChatCompletion(id='chatcmpl-9BHDTYvmThdpekNN3Mvl3TU5xL7Tq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The cost of a pizza salami is $10.99.', role='assistant', function_call=None, tool_calls=None))], created=1712474815, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=55, total_tokens=68))

The same can be achieved with LangChain

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.openai_functions import create_openai_fn_chain

In [12]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

template = """You are an AI chatbot having a conversation with a human.

Human: {human_input}
AI: """
prompt = PromptTemplate(input_variables=["human_input"], template=template)

chain = create_openai_fn_chain(functions, llm, prompt, verbose=True)

/Users/hyeonjinho/.pyenv/versions/3.9.18/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `create_openai_fn_chain` was deprecated in LangChain 0.1.1 and will be removed in 0.2.0. Use create_openai_fn_runnable instead.
  warn_deprecated(


In [13]:
chain.run("How much does pizza salami cost?")

/Users/hyeonjinho/.pyenv/versions/3.9.18/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
You are an AI chatbot having a conversation with a human.

Human: How much does pizza salami cost?
AI: 

> Finished chain.


{'pizza_name': 'Salami'}

We can also use Pydantic Classes instead of JSON Schemas

In [14]:
from pydantic import BaseModel, Field

class GetPizzaInfo(BaseModel):
    """Get name and price of a pizza of the restaurant."""

    pizza_name: str = Field(..., description="The name of the pizza, e.g. Salami")

pydantic_classes = [GetPizzaInfo]

In [15]:
chain = create_openai_fn_chain(pydantic_classes, llm, prompt, verbose=True)

In [16]:
chain.run("How much does pizza salami cost?")



> Entering new LLMChain chain...
Prompt after formatting:
You are an AI chatbot having a conversation with a human.

Human: How much does pizza salami cost?
AI: 

> Finished chain.


{'pizza_name': 'salami'}

We can also pass Functions directly. To pass Python function in directly, we'll want to make sure our parameters have type hints, we have a docstring, and we use Google Python style docstrings to describe the parameters.

In [17]:
def get_pizza_info(pizza_name: str) -> dict[str, str]:
    """
    Get name and price of a pizza of the restaurant.

    Args:
        pizza_name: The name of the pizza, e.g. Salami.

    Returns:
        Dict[str, str]: A dictionary containing the name and price of the pizza.
    """
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return pizza_info

In [18]:
chain = create_openai_fn_chain([get_pizza_info], llm, prompt, verbose=True)

In [19]:
chain.run("How much does pizza salami cost?")



> Entering new LLMChain chain...
Prompt after formatting:
You are an AI chatbot having a conversation with a human.

Human: How much does pizza salami cost?
AI: 

> Finished chain.


{'pizza_name': 'Salami'}